In [16]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [17]:
import sqlite3
import pandas as pd

db_path = 'real_estate_with_grade.db'

# 연결
conn = sqlite3.connect(db_path)

df_origin = pd.read_sql("SELECT * FROM house", conn)

conn.close()

In [18]:
df = df_origin

In [19]:
import pandas as pd
import numpy as np
import re

def parse_price_column(df, col='price'):
    def parse_price(text):
        try:
            text = str(text).replace(",", "").replace(" ", "")
            if "월세" in text or "단기임대" in text:
                match = re.search(r'(?:월세|단기임대)([\d억]+)[/\\](\d+)', text)
                if match:
                    deposit_raw = match.group(1)
                    if '억' in deposit_raw:
                        parts = deposit_raw.split('억')
                        deposit = int(parts[0]) * 10000
                        if len(parts) > 1 and parts[1].isdigit():
                            deposit += int(parts[1])
                    else:
                        deposit = int(deposit_raw)
                    deposit *= 10000
                    monthly = int(match.group(2))  * 10000
                    return deposit, monthly
            elif "전세" in text:
                match = re.search(r'전세\s*([\d억]+)', text)
                if match:
                    deposit_raw = match.group(1)
                    if '억' in deposit_raw:
                        parts = deposit_raw.split('억')
                        deposit = int(parts[0]) * 10000
                        if len(parts) > 1 and parts[1].isdigit():
                            deposit += int(parts[1])
                    else:
                        deposit = int(deposit_raw)
                    deposit *= 10000
                    return deposit, 0
        except:
            pass
        return None, None

    df[['deposit', 'monthly_rent']] = df[col].apply(lambda x: pd.Series(parse_price(x)))
    return df

def parse_area_column(df, col='area_size'):
    def parse_area(area_text):
        try:
            # '74.31㎡/49.92㎡(전용률67%)' 또는 '-㎡/38㎡' 처리
            matches = re.findall(r'([\d.]+)㎡', str(area_text))
            if len(matches) >= 1:
                return float(matches[-1])  # 전용면적만 추출
        except:
            pass
        return None

    df['전용면적'] = df[col].apply(parse_area)
    df['space'] = df['전용면적']/(3.3058)
    return df

def parse_rooms_column(df, col='rooms_count'):
    def parse_rooms(text):
        try:
            rooms, baths = re.findall(r'(\d+)', str(text))
            return int(rooms), int(baths)
        except:
            return None, None
    df[['rooms_count', 'bath_count']] = df[col].apply(lambda x: pd.Series(parse_rooms(x)))
    return df

def parse_floor_column(df, col='floor'):
    def parse_floor(text):
        try:
            text = str(text).replace(" ", "")
            parts = text.split('/')
            if len(parts) == 2:
                층_raw = parts[0]
                총층_match = re.search(r'(\d+)', parts[1])
                총층 = int(총층_match.group(1)) if 총층_match else None
                해당층_match = re.match(r'(\d+)', 층_raw)
                해당층 = int(해당층_match.group(1)) if 해당층_match else None
                return 해당층, 총층
        except:
            pass
        return None, None
    df[['floor', 'total_floor']] = df[col].apply(lambda x: pd.Series(parse_floor(x)))
    return df

### 2. 전처리 실행
df = parse_price_column(df)
df = parse_area_column(df)
df = parse_rooms_column(df)
df = parse_floor_column(df)

In [20]:
df.columns

Index(['house_id', 'title', 'price', 'address', 'floor', 'deposit_type',
       'management_fee', 'availabe_from', 'house_num', 'agent_comm',
       'agent_info', 'rooms_count', 'options', 'posted_at', 'gu', 'dong',
       'img_url', 'area_size', 'direction', 'built_date', 'parking',
       'building_type', 'house_feature', 'house_explanations', 'apt_name',
       'safety_grade', 'deposit', 'monthly_rent', '전용면적', 'space',
       'bath_count', 'total_floor'],
      dtype='object')

In [21]:
df['space'] = df['space'].fillna(0).round(0).astype(int)

In [22]:
df = df.drop(['전용면적'], axis=1)

In [23]:
df.columns

Index(['house_id', 'title', 'price', 'address', 'floor', 'deposit_type',
       'management_fee', 'availabe_from', 'house_num', 'agent_comm',
       'agent_info', 'rooms_count', 'options', 'posted_at', 'gu', 'dong',
       'img_url', 'area_size', 'direction', 'built_date', 'parking',
       'building_type', 'house_feature', 'house_explanations', 'apt_name',
       'safety_grade', 'deposit', 'monthly_rent', 'space', 'bath_count',
       'total_floor'],
      dtype='object')

In [24]:
pd.set_option('display.max_columns', None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50840 entries, 0 to 50839
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   house_id            50840 non-null  int64  
 1   title               50840 non-null  object 
 2   price               50840 non-null  object 
 3   address             50840 non-null  object 
 4   floor               35268 non-null  float64
 5   deposit_type        50840 non-null  object 
 6   management_fee      50840 non-null  object 
 7   availabe_from       50840 non-null  object 
 8   house_num           50840 non-null  object 
 9   agent_comm          50840 non-null  object 
 10  agent_info          50840 non-null  object 
 11  rooms_count         50742 non-null  float64
 12  options             50840 non-null  object 
 13  posted_at           50840 non-null  object 
 14  gu                  50840 non-null  object 
 15  dong                50840 non-null  object 
 16  img_

In [25]:
# management_fee, house_num, rooms_count, bath_count, floor, total_floor -> int로 변환
# management_fee, agent_comm(00만원형태) -> (10000)숫자로 변환 후 int

# parking -> boolean (1: 가능, 0: 불가능)

# posted_at -> datetime
# available_from, built_date -> datetime('정보 없음' 포함 : NULL)

# gu, dong -> string

In [26]:
# pandas에서 컬럼명 변경
df = df.rename(columns={'availabe_from': 'available_from'})

In [27]:
import numpy as np

In [28]:
import pandas as pd
import numpy as np
import re

# 1. management_fee: '8만원' → 80000 / '정보 없음' → NaN
df['management_fee'] = df['management_fee'].astype(str)
df['management_fee'] = df['management_fee'].apply(
    lambda x: np.nan if '정보없음' in x or '정보 없음' in x else (
        int(re.sub(r'[^\d]', '', x)) * 10000 if '만원' in x else pd.to_numeric(re.sub(r'[^\d]', '', x), errors='coerce')
    )
)
df['management_fee'] = pd.to_numeric(df['management_fee'], errors='coerce').astype('Int64')

# 2. agent_comm: '99만원' → 990000 / '정보 없음' → NaN
df['agent_comm'] = df['agent_comm'].astype(str)
df['agent_comm'] = df['agent_comm'].apply(
    lambda x: np.nan if '정보없음' in x or '정보 없음' in x else (
        int(re.sub(r'[^\d]', '', x)) * 10000 if '만원' in x else pd.to_numeric(re.sub(r'[^\d]', '', x), errors='coerce')
    )
)
df['agent_comm'] = pd.to_numeric(df['agent_comm'], errors='coerce').astype('Int64')

# 3. 숫자형 컬럼 → Int64 (NaN 포함)
cols_to_int = ['house_num', 'rooms_count', 'bath_count', 'floor', 'total_floor']
for col in cols_to_int:
    df[col] = pd.to_numeric(df[col], errors='coerce').round().astype('Int64')

# 4. parking 컬럼 → boolean (1: 가능, 0: 불가능)
df['parking'] = df['parking'].map({'가능': 1, '불가능': 0, '가능 ': 1, '불가': 0})
df['parking'] = df['parking'].fillna(0).astype(bool)

# 5. 날짜형 컬럼
df['posted_at'] = pd.to_datetime(df['posted_at'], errors='coerce').dt.date
df['built_date'] = df['built_date'].replace('정보 없음', np.nan)
df['built_date'] = pd.to_datetime(df['built_date'], errors='coerce').dt.date

# 6. 문자형 컬럼
df['gu'] = df['gu'].astype(str)
df['dong'] = df['dong'].astype(str)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50840 entries, 0 to 50839
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   house_id            50840 non-null  int64 
 1   title               50840 non-null  object
 2   price               50840 non-null  object
 3   address             50840 non-null  object
 4   floor               35268 non-null  Int64 
 5   deposit_type        50840 non-null  object
 6   management_fee      41930 non-null  Int64 
 7   available_from      50840 non-null  object
 8   house_num           50839 non-null  Int64 
 9   agent_comm          50673 non-null  Int64 
 10  agent_info          50840 non-null  object
 11  rooms_count         50742 non-null  Int64 
 12  options             50840 non-null  object
 13  posted_at           50840 non-null  object
 14  gu                  50840 non-null  object
 15  dong                50840 non-null  object
 16  img_url             50

In [30]:
import sqlite3

conn = sqlite3.connect('cleaned_real_estate.db')
df.to_sql('house', conn, index=False, if_exists='replace')
conn.close()